Make ROI Spheres from Coordinates

In [ ]:
from nilearn import image
import numpy as np
import pandas as pd
from numpy.linalg import inv
from tqdm import tqdm
from glob import glob
from nimlab import datasets

By default, this notebook assumes that the coordinates list is in MNI space. If it is already in voxel space, 
add `voxel_coord = True` to the call to `make_spheres` in the last cell 

Load source dataframe
---

In [ ]:
src_df = pd.read_csv('./my_coords.csv')
src_df.head()

Specify which columns in the csv file to use as subject identifiers and coordinates in the dataframe:

In [ ]:
coord_df = pd.DataFrame({
    'subject_id': src_df['subject_id'],
    'mni_x': src_df['mni_x'],
    'mni_y': src_df['mni_y'],
    'mni_z': src_df['mni_z'],
})

coord_df.head()

In [ ]:
def create_bin_sphere(arr_size, center, r):
    # https://stackoverflow.com/questions/53326570/how-to-create-sphere-inside-a-ndarray-python?noredirect=1&lq=1
    coords = np.ogrid[:arr_size[0], :arr_size[1], :arr_size[2]]
    distance = np.sqrt((coords[0] - center[0])**2 + (coords[1]-center[1])**2 + (coords[2]-center[2])**2) 
    return 1*(distance <= r)

def make_sphere(coord, brain_mask, radius, voxel_coord = False):
    #Transform from MNI coordinates to voxelwise(matrix) coords
    if voxel_coord == False:
        inv_affine = inv(brain_mask.affine)
        
        trans_raw_coord = image.coord_transform(coord[0], coord[1], coord[2], inv_affine)
        trans_coord = round(trans_raw_coord[0]), round(trans_raw_coord[1]), round(trans_raw_coord[2])

    else:
        trans_coord = coord

    bin_sphere = create_bin_sphere(brain_mask.shape, trans_coord, radius)
    sphere_img = image.new_img_like(brain_mask, bin_sphere)
    return sphere_img
    

Input required here:
---

In [ ]:
# Specify output directory:
outdir = './output_folder'

# Specify radius in voxels (NOT millimeters):
radius = 2

# Specify MNI brain mask to get affine transfrom from:
mni_brain = './MNI152_T1_2mm_brain_mask_dil.nii.gz'

for c in tqdm(coord_df.itertuples()):

    sphere = make_sphere((c.mni_x, c.mni_y, c.mni_z), mni_brain, radius)
    sphere.to_filename(outdir + '/' + c.subject_id + 
                       '_' + str(c.mni_x).replace('.','d') + 
                       '_' + str(c.mni_y).replace('.','d') + 
                       '_' + str(c.mni_z).replace('.','d') + 
                       '.nii.gz')